In [15]:
import pandas as pd
from rapidfuzz import fuzz, process
import json
import numpy as np
from sentence_transformers import SentenceTransformer
from keywordmaker import KeyWordMaker

In [68]:
import torch
import transformers

print(torch.__version__)
print(transformers.__version__)

2.7.0+cpu
4.51.3


In [16]:
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

In [17]:

with open(r"C:\Users\jarja\Desktop\Studia\teg\TEG_Final_Project\MOJtest\acts_backup.json", "r", encoding="utf-8") as f:
    data = [json.loads(line) for line in f]



In [18]:
df=pd.DataFrame(data)

In [86]:
print(np.shape(df))


(28807, 32)


In [19]:

collumtoTransfer=['title','keywords','type','address','displayAddress',"texts"]
newdf = pd.DataFrame()
newdf[collumtoTransfer]=df[collumtoTransfer]


In [20]:
fileNames = []
fileTypes = []

for files in newdf["texts"]:
    pliki_T = [t["fileName"] for t in files if t.get("type") == "T"]
    pliki_O = [t["fileName"] for t in files if t.get("type") == "O"]
    if pliki_T:
        fileNames.append(pliki_T)       # lista plików T
        fileTypes.append("T")
    elif pliki_O:
        fileNames.append(pliki_O)       # lista plików O
        fileTypes.append("O")
    else:
        fileNames.append([])            # brak plików T/O
        fileTypes.append("brak")

# Dodanie kolumn do DataFrame
newdf["fileNames"] = fileNames
newdf["typeFile"] = fileTypes
print(fileNames)



[['D20250611L.pdf'], ['D20250610L.pdf'], ['D20250609.pdf'], ['D20250608L.pdf'], ['D20250607L.pdf'], ['D20250606.pdf'], ['D20250605.pdf'], ['D20250604.pdf'], ['D20250603L.pdf'], ['D20250602L.pdf'], ['D20250600.pdf'], ['D20250599.pdf'], ['D20250598.pdf'], ['D20250597.pdf'], ['D20250596.pdf'], ['D20250595L.pdf'], ['D20250594L.pdf'], ['D20250592L.pdf'], ['D20250591.pdf'], ['D20250590L.pdf'], ['D20250589.pdf'], ['D20250588.pdf'], ['D20250587.pdf'], ['D20250586.pdf'], ['D20250585.pdf'], ['D20250584.pdf'], ['D20250583.pdf'], ['D20250582.pdf'], ['D20250581L.pdf'], ['D20250580.pdf'], ['D20250579.pdf'], ['D20250578.pdf'], ['D20250577.pdf'], ['D20250576.pdf'], ['D20250575.pdf'], ['D20250574.pdf'], ['D20250573L.pdf'], ['D20250572L.pdf'], ['D20250571.pdf'], ['D20250570L.pdf'], ['D20250569.pdf'], ['D20250568L.pdf'], ['D20250567L.pdf'], ['D20250566L.pdf'], ['D20250565L.pdf'], ['D20250564.pdf'], ['D20250563.pdf'], ['D20250562.pdf'], ['D20250561L.pdf'], ['D20250560L.pdf'], ['D20250559L.pdf'], ['D202505

In [21]:
print(newdf["fileNames"])

0        [D20250611L.pdf]
1        [D20250610L.pdf]
2         [D20250609.pdf]
3        [D20250608L.pdf]
4        [D20250607L.pdf]
               ...       
28802     [M19470417.pdf]
28803     [M19460008.pdf]
28804     [M19390059.pdf]
28805     [M19340011.pdf]
28806     [M19340010.pdf]
Name: fileNames, Length: 28807, dtype: object


In [21]:
# Usuwamy puste listy (len == 0)
newdf_ = newdf[newdf["keywords"].apply(lambda x: isinstance(x, list) and len(x) > 0)]
newdf_ = newdf_.reset_index(drop=True)

print(newdf_["keywords"])
numpyarray=newdf_["keywords"]

0        [finansowa gospodarka, nadzór, sprawozdawczość...
1        [energetyczna gospodarka, energetyka, energety...
2        [żołnierz zawodowy, mieszkania funkcyjne, wojs...
3                                             [farmaceuci]
4        [przewozowe prawo, towary niebezpieczne, zezwo...
                               ...                        
28273                                           [amnestia]
28274                                           [amnestia]
28275            [wydawnictwa i publikacje, prasowe prawo]
28276                         [izby rzemieślnicze, statut]
28277                         [izby rzemieślnicze, statut]
Name: keywords, Length: 28278, dtype: object


In [22]:
texts = [' '.join(kw_list) for kw_list in newdf["keywords"]]

print(texts)

['finansowa gospodarka nadzór sprawozdawczość elektroniczny instrument płatniczy działalność gospodarcza waluta agencje płatnicze usługi rachunkowość banki bankowe prawo stopa procentowa', 'energetyczna gospodarka energetyka energetyczne prawo', 'żołnierz zawodowy mieszkania funkcyjne wojsko lokalowe prawo', 'farmaceuci', '', 'przewozowe prawo towary niebezpieczne zezwolenia transport wojsko', 'kwalifikacje', 'farmaceutyczne prawo farmaceutyczne środki dokumenty leki i artykuły sanitarne dokumentacja medyczna', '', 'energetyczna gospodarka energetyka energetyczne prawo', 'umowy międzynarodowe ochrona środowiska', 'komornicy opłaty', 'narkomania', 'karne postępowanie cywilne postępowanie międzynarodowe prawo prywatne', 'żołnierz zawodowy dodatki do wynagrodzeń uposażenie', 'gry losowe i zakłady wzajemne', 'akta stanu cywilnego', 'instrumenty finansowe oferty', 'dyscyplinarne postępowanie rzecznicy patentowi aplikacja', 'rybacy morscy rybołówstwo morskie ryby', 'warunki sanitarne żywność

In [23]:

embededkeywors=model.encode(texts)



In [ ]:
print(np.shape(embededkeywors))
avg_embedding = np.mean(embededkeywors, axis=0)
newdf["embeded"] = list(embededkeywors)

toche=['prawo pracy', 'umowa o prac�', 'pracodawca', 'dyscyplina pracy', 'dyscyplinarne post�powanie', 'kodeks pracy']

embeded=model.encode(toche)

toche_=np.mean(embeded, axis=0)

(28807, 384)


In [26]:
from sklearn.metrics.pairwise import cosine_similarity
def search_similar_documents(query_keywords, df, model, top_k=50):
    # Jeśli query_keywords jest listą słów, można zrobić join na string
    if isinstance(query_keywords, list):
        query_text = " ".join(query_keywords)
    else:
        query_text = query_keywords

    query_embedding = model.encode([query_text])[0]  
    embeddings_matrix = np.vstack(df["embeded"].values)  

    similarities = cosine_similarity([query_embedding], embeddings_matrix)[0]
    df = df.copy()  
    df["similarity"] = similarities
    return df.sort_values(by="similarity", ascending=False).head(top_k)

# Użycie
results = search_similar_documents(toche, newdf, model)
print(results[["keywords", "similarity", "title"]])

                                                keywords  similarity  \
18632  [umowa o pracę, porządkowanie prawa, prawo pra...    0.920488   
12759                      [umowa o pracę, kodeks pracy]    0.919477   
17064                      [umowa o pracę, kodeks pracy]    0.919477   
14311  [praca, pracodawca, prawo pracy, zaświadczenia...    0.909734   
21996  [pracodawca, prawo pracy, kodeks pracy, zatrud...    0.904205   
21571                [porządkowanie prawa, kodeks pracy]    0.894928   
22305                [porządkowanie prawa, kodeks pracy]    0.894928   
18350                [porządkowanie prawa, kodeks pracy]    0.894928   
19209                [porządkowanie prawa, kodeks pracy]    0.894928   
20654                [porządkowanie prawa, kodeks pracy]    0.894928   
20489                [porządkowanie prawa, kodeks pracy]    0.894928   
19885                [porządkowanie prawa, kodeks pracy]    0.894928   
19886  [porządkowanie prawa, kodeks pracy, kodeks pos...    0.88

In [32]:
print((results["title"]))
# "texts": [{"fileName": "text.html", "type": "H"}, {"fileName": "D20081113.pdf", "type": "O"}

18632    Wyrok Trybunału Konstytucyjnego z dnia 2 grudn...
12759    Ustawa z dnia 13 maja 2016 r. o zmianie ustawy...
17064    Ustawa z dnia 5 stycznia 2011 r. o zmianie ust...
14311    Obwieszczenie Ministra Pracy i Polityki Społec...
21996    Ustawa z dnia 13 marca 2003 r. o szczególnych ...
21571    Wyrok Trybunału Konstytucyjnego z dnia 24 lute...
22305    Wyrok Trybunału Konstytucyjnego z dnia 18 list...
18350    Wyrok Trybunału Konstytucyjnego z dnia 31 marc...
19209    Wyrok Trybunału Konstytucyjnego z dnia 27 list...
20654    Wyrok Trybunału Konstytucyjnego z dnia 28 czer...
20489    Wyrok Trybunału Konstytucyjnego z dnia 4 paźdz...
19885    Wyrok Trybunału Konstytucyjnego z dnia 24 paźd...
19886    Wyrok Trybunału Konstytucyjnego z dnia 23 paźd...
10898    Ustawa z dnia 9 listopada 2017 r. o zmianie ni...
18953    Ustawa z dnia 11 kwietnia 2008 r. o zmianie us...
20194    Ustawa z dnia 7 kwietnia 2006 r. o informowani...
3673     Rozporządzenie Ministra Rodziny i Polityki Spo.

In [28]:
from ai_errors import BadAiApiRes, BadUserPrompt, AiAgentError
from openai import OpenAI
import os
import ai_prompts as aiP


client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

def check_title(title,prompt):
    system_prompt = aiP.prompt_cheack(title,prompt)
                
    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "system", "content": system_prompt}])
    return response.choices[0].message.content
    

In [29]:
arrat=[]
for i, row in results.iterrows():
    t=check_title(row["title"],"Czy pracodawca może rozwiązać umowę o pracę bez wypowiedzenia z powodu nieusprawiedliwionej nieobecności pracownika?")
    if t=="True":
        arrat.append(row)
        print("HU")

In [31]:
for a in arrat:
    print(a["texts"])
    print(a["address"])

In [ ]:
#https://isap.sejm.gov.pl/isap.nsf/download.xsp/WDU20140001632/O/D20141632.pdf

In [13]:
import requests


def download_pdf(url):
    headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
    }


    r = requests.get(url, headers=headers)
    if r.status_code == 200:
        with open("./pobrane_pdf/plik.pdf", "wb") as f:
            f.write(r.content)
    else:
        print("Błąd pobierania", r.status_code)
download_pdf("https://isap.sejm.gov.pl/isap.nsf/download.xsp/WDU20140001632/O/D20141632.pdf")

In [9]:
import pdfplumber
import re
from sentence_transformers import SentenceTransformer
def extract_text_from_pdf_starting_from_paragraph(pdf_path, start_pattern=r'§\s*1'):
    with pdfplumber.open(pdf_path) as pdf:
        full_text = ""
        for page in pdf.pages:
            text = page.extract_text()
            if text:
                full_text += text + "\n"
        return full_text.strip()
    # match = re.search(start_pattern, full_text)
    # if match:
    #     start_index = match.start()
        # return full_text[start_index:].strip()
    # else:
        
from langchain.text_splitter import RecursiveCharacterTextSplitter

def split_text(text, max_words_=300,chunk_overlap_=75):
    splitter = RecursiveCharacterTextSplitter(
    chunk_size=max_words_,          
    chunk_overlap=chunk_overlap_,         
    separators=[r"\n\n", r"\n", r"Art\. ", r"§ "]      
)
    chunks = splitter.split_text(text)
    print(chunks)
    return chunks
def get_text_embeddings(text_chunks, model_name='all-MiniLM-L6-v2'):
    model = SentenceTransformer(model_name)
    embeddings = model.encode(text_chunks, show_progress_bar=True)
    return embeddings

# Użycie:
pdf_file = "plik.pdf"
tekst = extract_text_from_pdf_starting_from_paragraph(pdf_file)

if tekst:
    chunks = split_text(tekst)
    embeddings = get_text_embeddings(chunks)
    print(f"Liczba fragmentów: {len(chunks)}")
    print(f"Embedding pierwszego fragmentu: {embeddings[0]}")
else:
    print("Nie znaleziono paragrafu zaczynającego się od § 1.")

['DZIENNIK USTAW\nRZECZYPOSPOLITEJ POLSKIEJ\nWarszawa, dnia 24 listopada 2014 r.\nPoz. 1632\nOBWIESZCZENIE\nMINISTRA PRACY I POLITYKI SPOŁECZNEJ\nz dnia 16 września 2014 r.\nw sprawie ogłoszenia jednolitego tekstu rozporządzenia Ministra Pracy i Polityki Socjalnej w sprawie sposobu\nusprawiedliwiania nieobecności w pracy oraz udzielania pracownikom zwolnień od pracy\n1. Na podstawie art. 16 ust. 3 ustawy z dnia 20 lipca 2000 r. o ogłaszaniu aktów normatywnych i niektórych innych\naktów prawnych (Dz. U. z 2011 r. Nr 197, poz. 1172 i Nr 232, poz. 1378) ogłasza się w załączniku do niniejszego\nobwieszczenia jednolity tekst rozporządzenia Ministra Pracy i Polityki Socjalnej z dnia 15 maja 1996 r. w sprawie\nsposobu usprawiedliwiania nieobecności w pracy oraz udzielania pracownikom zwolnień od pracy (Dz. U. Nr 60,\npoz. 281), z uwzględnieniem zmian wprowadzonych:\n1) rozporządzeniem Ministra Pracy i Polityki Społecznej z dnia 29 listopada 2007 r. zmieniającym rozporządzenie\nw sprawie sposo

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Liczba fragmentów: 33
Embedding pierwszego fragmentu: [-4.32577506e-02  1.00668132e-01 -8.05314630e-02 -2.54060104e-02
 -9.44975838e-02 -2.03639027e-02  1.33140646e-02  4.49526608e-02
 -6.61303252e-02  3.88445482e-02  3.00107412e-02  4.84634936e-02
  8.34972784e-03  3.76409553e-02 -3.20285298e-02  1.48226572e-02
 -1.02658560e-02  5.49789555e-02 -7.65574574e-02  7.58880004e-02
  6.49028346e-02 -2.31766701e-02  7.79124722e-02 -4.51708101e-02
  1.17390184e-03  1.07322959e-02 -1.79954879e-02  4.92018275e-02
  3.28095742e-02 -2.48584934e-02  1.42509993e-02  7.21674412e-02
  4.23832908e-02 -8.73831958e-02  1.33791298e-01  8.01649969e-03
  3.32038030e-02 -1.24474205e-02 -4.29594070e-02  7.45190680e-02
 -4.07045670e-02 -1.02692822e-04 -9.15338919e-02 -6.91489410e-03
 -9.07838717e-02 -1.21931201e-02 -7.21511766e-02  3.07362266e-02
 -9.05665681e-02 -1.94302388e-02 -6.54155239e-02 -4.41784337e-02
  2.31073368e-02 -5.08280173e-02 -2.14685388e-02 -1.48957744e-01
 -4.69589047e-02  5.19315898e-02 -4.

In [12]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
def search_in_document(question, chunks, embeddings, model_name='all-MiniLM-L6-v2', top_k=10):
    model = SentenceTransformer(model_name)
    question_embedding = model.encode([question])[0]  # embedding pytania

    # Oblicz podobieństwo kosinusowe
    similarities = cosine_similarity([question_embedding], embeddings)[0]

    # Znajdź top k indeksów o największym podobieństwie
    top_indices = similarities.argsort()[-top_k:][::-1]

    # Zwróć top k fragmentów i ich score
    results = [(chunks[i], similarities[i]) for i in top_indices]
    return results

# Przykład użycia:
question = "Czy pracodawca może rozwiązać umowę o pracę bez wypowiedzenia z powodu nieusprawiedliwionej nieobecności pracownika?"
results = search_in_document(question, chunks, embeddings)

for i, (text, score) in enumerate(results):
    print(f"Fragment #{i+1} (similarity: {score:.4f}):\n{text}\n{'-'*40}")

Fragment #1 (similarity: 0.7577):
§ 1. Przyczynami usprawiedliwiającymi nieobecność pracownika w pracy są zdarzenia i okoliczności określone prze-
pisami prawa pracy, które uniemożliwiają stawienie się pracownika do pracy i jej świadczenie, a także inne przypadki
niemożności wykonywania pracy wskazane przez pracownika i uznane przez pracodawcę za usprawiedliwiające nieobec-
ność w pracy.

----------------------------------------
Fragment #2 (similarity: 0.7558):
§ 5. Pracodawca jest obowiązany zwolnić od pracy pracownika wezwanego do osobistego stawienia się przed orga-
nem właściwym w zakresie powszechnego obowiązku obrony na czas niezbędny w celu załatwienia sprawy będącej
przedmiotem wezwania.
----------------------------------------
Fragment #3 (similarity: 0.7249):
§ 8. Pracodawca jest obowiązany zwolnić od pracy pracownika na czas niezbędny do wzięcia udziału w posiedzeniu
komisji pojednawczej w charakterze członka tej komisji. Dotyczy to także pracownika będącego stroną lub świa